In [1]:
#r "nuget:Serilog.Formatting.Compact.Reader,1.0.3"
#r "nuget:System.Linq.Async,4.0.0"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package System.Linq.Async version 4.0.0

Installed package Serilog.Formatting.Compact.Reader version 1.0.3

In [4]:
using System.IO;
using Serilog.Events;
using Serilog.Formatting.Compact.Reader;
using System.Collections.Generic;
using System.Linq;

In [33]:
public async IAsyncEnumerable<(DateTimeOffset x, double bestFps, double worstFps, double averagefps)> GetBestFrameCount()
{
    var compactFile = new FileInfo(@"C:\Users\vaido\SourceCode\MrBabyData\FrameRateAnalysis\RogueTD_PerformanceTestScene_CreepSpawning_AfterPooling.clef");
    using(var stream = compactFile.OpenText())
    {
        while(stream.Peek() > 0)
        {
            var logevent = Serilog.Formatting.Compact.Reader.LogEventReader.ReadFromString(await stream.ReadLineAsync());
            if(logevent.Properties["@i"].ToString() != "\"0ed99f63\"")
                continue;
            yield return (
                logevent.Timestamp,
                1/double.Parse(logevent.Properties["bestDuration"].ToString()),
                1/double.Parse(logevent.Properties["worstDuration"].ToString()),
                double.Parse(logevent.Properties["framesInSample"].ToString()) / double.Parse(logevent.Properties["sampleDuration"].ToString())
            );
        }
    }
}

public struct PlotDatum{
    public double x {get; set;}
    public double y0 {get; set;}
    public double y1 {get; set;}
    public double y2 {get; set;}
}

var readData = await GetBestFrameCount().ToListAsync();

// convert our data to format plot format
var firstTime = readData.First().x;
var dataOp1 = readData.Select(i => new PlotDatum
    {
        x = (i.x - firstTime).TotalSeconds, 
        y0 = i.worstFps, 
        y1 = i.bestFps,
        y2 = i.averagefps
    })
    .ToList();

// data close load and unload time of the scene might be incorrect and unrepresentitive.  Trim some data from begining and end.
var data = dataOp1
    .SkipWhile(i => i.x <= 5)
    .TakeWhile( i => i.x <= dataOp1.Last().x - 5);
    data

index,x,y0,y1,y2
0,6.0021932,147.5143826523086,316.2255320494577,271
1,8.0042822,132.69286908521536,284.6407833314357,249.5
2,10.006125,50.76863716670389,284.2443364315966,250
3,12.0057701,175.297128633033,283.5351158240948,255.5
4,14.0093336,139.09560040615915,264.85155070582937,227
5,16.0097102,48.45923851152603,253.41476394414738,221.5
6,18.0142298,159.8823266076168,255.12807429329527,218.5
7,20.0160294,125.80831844601565,244.39122146732493,201.5
8,22.0254197,42.29384920551004,237.64823308538703,200
9,24.0274305,103.97604391948094,252.21953188054883,214.5


In [30]:
public async IAsyncEnumerable<(DateTimeOffset x, string poolName, string spawnedObjectName)> GetEventData()
{
    var compactFile = new FileInfo(@"C:\Users\vaido\SourceCode\MrBabyData\FrameRateAnalysis\RogueTD_PerformanceTestScene_CreepSpawning_AfterPooling.clef");
    using(var stream = compactFile.OpenText())
    {
        while(stream.Peek() > 0)
        {
            var logevent = Serilog.Formatting.Compact.Reader.LogEventReader.ReadFromString(await stream.ReadLineAsync());
            if(logevent.Properties["@i"].ToString() != "\"defdd1ef\"")
                continue;
            yield return (
                logevent.Timestamp,
                logevent.Properties["Pool name"].ToString(),
                logevent.Properties["ObjectName"].ToString()
            );
        }
    }
}

public struct PlotEventData{
    public double x {get; set;}
}


var readData = await GetEventData().ToListAsync();
var firstTime = readData.First().x;
var eventdataOp1 = readData.Select(i => new PlotEventData
    {
        x = (i.x - firstTime).TotalSeconds, 
    })
    .ToList();
var eventData = eventdataOp1
    .SkipWhile(i => i.x <= 5)
    .TakeWhile( i => i.x <= dataOp1.Last().x - 5);

    eventData


index,x
0,6.5556076
1,6.6593295
2,6.7620555
3,6.8647805
4,6.9665086
5,7.0668069
6,7.170529
7,7.2732552
8,7.3728835
9,7.4736146


In [138]:
#!javascript
notebookScope.plot = (svgSelector, dataVariable, eventDataVariableName) => {

    let dtreeLoader = interactive.configureRequire({
        paths: {
            d3: "https://d3js.org/d3.v6.min"
        }
    });
    
    dtreeLoader(["d3"], function (d3) {
    
      var buildFrameRatePlot = (svg, width, height, DenseData, SparseData) => {
        var layoutMeasures = {
          width : width,  
          height : height,
          plotPadding: [48,48,48,48], // top, right, bottom, left
          get plotWidth()
          {
            return layoutMeasures.width - layoutMeasures.plotPadding[1] - layoutMeasures.plotPadding[3]
          },
          get plotHeight()
          {
            return layoutMeasures.height - layoutMeasures.plotPadding[0] - layoutMeasures.plotPadding[2]
          }
        }
     
        // clean clear our svg
        svg.selectAll("g").remove();
      
        // plot container
        let container = svg
        .append("g")
        .attr("height", layoutMeasures.plotHeight)
        .attr("width", layoutMeasures.plotWidth)
        .attr("transform", `translate(${layoutMeasures.plotPadding[0]}, ${layoutMeasures.plotPadding[3]})`)
        .style("color", "black");
    
        container.append("rect")
          .attr("height", layoutMeasures.plotHeight)
          .attr("width", layoutMeasures.plotWidth)
          .style("fill", "#F7F7F7")
      
        var y = d3.scaleLinear()
          .domain([0, d3.max(DenseData.flatMap(i => [i.y0, i.y1]))])
          .nice()
          .range([layoutMeasures.plotHeight, 0]);
        var x = d3.scaleLinear()
          .domain([0, d3.max(DenseData, i => i.x)])
          .range([0, layoutMeasures.plotWidth])
        var yAxis = g => g
            .attr("transform",`translate(0,0)`)
            .call(d3.axisLeft(y))
        var xAxis = g => g
            .attr("transform",`translate(0,${layoutMeasures.plotHeight})`)
            .call(d3.axisBottom(x))
        container.append("g")
            .call(yAxis);
        container.append("g")
            .call(xAxis);
      
      // Area - difference between best and worst framerate
        var areaDifference = d3.area()
          .curve(d3.curveLinear)
          .defined(d => !isNaN(d.x))
          .defined(d => !isNaN(d.y0))
          .x(d => x(d.x))
          .y0(d => y(d.y0))
          .y1(d => y(d.y1));
        container.append("path")
          .datum(DenseData)
          .attr("fill", "steelBlue")
          .attr("d", areaDifference);
      
        // Line - best framerate in sample
        var linePoints_BestFramerate = d3.line()
            .x(d => x(d.x))
            .y(d => y(d.y0))
            (DenseData);
        container.append('path')
          .attr('fill', 'none')
          .attr('stroke', 'red')
          .attr('stroke-width', 2)
          .attr("stroke-linejoin", "round")
          .attr('d', linePoints_BestFramerate);
      
        // Line - worst Framerate in sample
        var linePoints_WorstFramerate = d3.line()
            .x(d => x(d.x))
            .y(d => y(d.y1))
            (DenseData);
        container.append('path')
          .attr('fill', 'none')
          .attr('stroke', 'green')
          .attr('stroke-width', 2)
          .attr("stroke-linejoin", "round")
          .attr('d', linePoints_WorstFramerate);
      
        // Line - average framerate in sample
        var linePoints_AverageFramerate = d3.line()
            .x(d => x(d.x))
            .y(d => y(d.y2))
            (DenseData);
      console.log(linePoints_AverageFramerate);
        container.append('path')
          .attr('fill', 'none')
          .attr('stroke', 'yellow')
          .attr('stroke-width', 2)
          .attr("stroke-linejoin", "round")
          .attr('d', linePoints_AverageFramerate);
      
      // Event lines
      container.selectAll("EventLines")
        .data(SparseData)
        .enter()
          .append('line')
          .attr('fill', 'none')
          .attr('stroke', 'black')
          .attr('stroke-width', 1)
          .attr('x1', i => x(i.x))
          .attr('x2', i => x(i.x))
          .attr('y1', 0)
          .attr('stroke', 'purple')
          .attr('opacity', 0.25)
          .attr('y2',layoutMeasures.plotHeight);
    
      // Plot title
      svg.append('text')
        .text("Frame Rate Over Time")
        .attr('x', layoutMeasures.width / 2)
        .attr('y', 24)
        .attr('dominant-baseline', 'middle')
        .attr("text-anchor", "middle")
        .style('font-size', 24)
      
      // X-axis title
      container.append('text')
        .text('Time since scene start (seconds)')
        .attr('dominant-baseline', 'middle')
        .attr('text-anchor', 'middle')
        .attr('x', layoutMeasures.plotWidth / 2)
        .attr('y', layoutMeasures.plotHeight + 36)
      
      // Y-axis title
      container
        .append('g')
        .attr('transform', `translate(-36, ${layoutMeasures.plotHeight / 2})`)
        .append('text')
          .text('Frames per second')
          .attr('dominant-baseline', 'middle')
          .attr('text-anchor', 'middle')
          .attr('transform', 'rotate(-90)')
        
      // Legend
      var legendData = [
        {label:'FPS based on worst refresh', color: 'green'},
        {label:'FPS based on average refresh', color:'yellow'},
        {label:'FPS based on best refresh', color:'red'}
      ]
      var legend =svg
        .append('g')
        .attr('transform',`translate(${layoutMeasures.width - layoutMeasures.plotPadding[1] - 240}, 0)`)
      var legendScale = d3
        .scaleOrdinal(legendData.map(i=>i.color))
        .domain(legendData.map(i=>i.label));
      legend.selectAll("boxes")
        .data(legendData)
        .enter()
        .append('rect')
          .attr('fill',i => i.color)
          .attr('y', (d,i) => i * 24 + 7)
          .attr('width', 50)
          .attr('height', 14);
      legend.selectAll("labels")
        .data(legendData)
        .enter()
        .append('text')
          .attr('y', (d,i) => i * 24 + 14)
          .attr('x', 60)
          .attr('dominant-baseline', 'middle')
          .style('font-size', 14)
          .text(d=>d.label);
    }
    
    var data = interactive.csharp.getVariable(dataVariable);
    var eventData = interactive.csharp.getVariable(eventDataVariableName);
    
    data.then( (d1) =>
    {
        eventData.then((d2)=>
        {
            console.log(d2);
            var mySvg = d3.select(svgSelector);
            buildFrameRatePlot(mySvg, 1200, 400, d1, d2);
        })
    })
});
}

In [139]:
#!html
<svg id="dataPlot1" width=1400 height=600  style="background-color:white"></svg>

#!js
notebookScope.plot("svg#dataPlot1", "data", "eventData");

In [103]:
#!html
